<a href="https://colab.research.google.com/github/tianzhanyuan/IncompleteDiscreteChoice/blob/main/Estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Estimation

The goal of this note is to estimate the identified set using a method by Chernozhukov, Hong, and Tamer (2007) (CHT below). Their idea is to define a sample criterion function
\begin{align}
\hat Q_n(\theta)
\end{align}
and use its level set as an estimator of $\Theta_I(P)$.

Below, we
- generate data
- compute conditional choice probabilities (CCPs)
- compare CCPs with the sharp lower bound
- define a sample criterion function

# Data generation

Suppose a sample is generated from an entry game.
For this, let's simulate data from the following game.

|  | $Y_2=0$ | $Y_2=1$ |
|----------|----------|----------|
| Enter ($Y_1=0$)  | $(0,0)$   | $(0,X_2'\beta_2+U_2)$   |
| Do not enter ($Y_1=1$)  | $(X_1'\beta_1+U_1, 0)$  | $(X_1'\beta_1+\Delta_1+U_1,X_2'\beta_2+\Delta_2+U_2)$  |

We set
- $X=(X_1,X_2)$ where $X_{j},j=1,2$ are independent Bernoulli random variables.
- $\beta_1$ = 0.75
- $\beta_2$ = 0.25
- $\Delta_1$ = -0.5
- $\Delta_2$ = -0.5
- $\rho$ = 0.5 ($U_1$ and $U_2$'s correlation)

The following code generates data.

In [21]:
import numpy as np

def simulate_y(n, beta1, beta2, delta1, delta2, rho, Y_nodes, seed=None):
    """
    Simulate Y based on given parameters and regions, and store X and Y values.

    Parameters:
    n (int): Number of simulations
    rho (float): Correlation coefficient between U1 and U2
    beta1 (float): Coefficient for U1
    beta2 (float): Coefficient for U2
    delta1 (float): Threshold adjustment for region01
    delta2 (float): Threshold adjustment for region10
    Y_nodes (list of tuples): Possible values for Y
    seed (int, optional): Seed for the random number generator

    Returns:
    tuple: Two numpy arrays, X_vals and Y, both of shape (n, 2)
    """
    if seed is not None:
        np.random.seed(seed)

    # Covariance matrix for the bivariate normal distribution
    cov = [[1, rho], [rho, 1]]

    # Storage for the results
    Y = np.zeros((n, 2))
    X_vals = np.zeros((n, 2))

    # Simulation
    for i in range(n):
        # Generate U from a bivariate normal distribution
        U = np.random.multivariate_normal([0, 0], cov)

        # Generate X from independent Bernoulli distributions
        X = np.random.binomial(1, 0.5, 2)
        #X = np.random.standard_normal(2)
        X_vals[i] = X

        # Calculate the threshold values for regions
        threshold1_00 = -X[0] * beta1
        threshold2_00 = -X[1] * beta2
        threshold1_01 = -X[0] * beta1 - delta1
        threshold2_10 = -X[1] * beta2 - delta2

        # Determine the region and assign Y
        if U[0] <= threshold1_00 and U[1] <= threshold2_00:
            Y[i] = Y_nodes[0]
        elif U[0] <= threshold1_01 and U[1] >= threshold2_00 and not (U[0] >= threshold1_00 and U[1] <= threshold2_10):
            Y[i] = Y_nodes[1]
        elif U[0] >= threshold1_00 and U[1] <= threshold2_10 and not (U[0] <= threshold1_01 and U[1] >= threshold2_00):
            Y[i] = Y_nodes[2]
        elif U[0] >= threshold1_01 and U[1] >= threshold2_10:
            Y[i] = Y_nodes[3]
        elif (U[0] <= threshold1_01 and U[1] >= threshold2_00) and (U[0] >= threshold1_00 and U[1] <= threshold2_10):
            Y[i] = Y_nodes[np.random.choice([1, 2])]

    return X_vals, Y

# Example usage
n = 1000
beta1 = 0.75
beta2 = 0.25
delta1 = -0.5
delta2 = -0.5
rho = 0.5
theta_true = [beta1, beta2, delta1, delta2, rho]
Y_nodes = [(0, 0), (0, 1), (1, 0), (1, 1)]
seed = 123

# Simulate the values
X, Y = simulate_y(n, rho, beta1, beta2, delta1, delta2, Y_nodes,seed=seed)
print(Y)

[[1. 0.]
 [1. 1.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 0.]
 [0. 1.]]


# Computing CCP
Now let's compute the sample conditional choice probabilities, which we can use to construct a sample criterion function.

In [22]:
!pip install scikit-optimize
!git clone https://github.com/hkaido0718/IncompleteDiscreteChoice.git
import IncompleteDiscreteChoice.idclib as idc

fatal: destination path 'IncompleteDiscreteChoice' already exists and is not an empty directory.


The idc library has a function called calculate_ccp. For this, we should pass the data ($Y,X$) and their support.

In [23]:
conditional_probabilities,ccp_array, Px, X_supp = idc.calculate_ccp(Y,X, Y_nodes)

# Print the conditional probabilities for the specified X support
for x in list(conditional_probabilities.keys())[:5]:
    print(f"P(Y|X={x}) = {conditional_probabilities[x]}")

# Support of X and X's marginal distribution over it
print(X_supp)
print(Px)

# This is the CCP matrix (sorted according to X_supp)
print(ccp_array)

P(Y|X=(np.float64(0.0), np.float64(1.0))) = {(0, 0): 0.08171206225680934, (0, 1): 0.3852140077821012, (1, 0): 0.26459143968871596, (1, 1): 0.26848249027237353}
P(Y|X=(np.float64(1.0), np.float64(1.0))) = {(0, 0): 0.05982905982905983, (0, 1): 0.15384615384615385, (1, 0): 0.33760683760683763, (1, 1): 0.44871794871794873}
P(Y|X=(np.float64(0.0), np.float64(0.0))) = {(0, 0): 0.1857707509881423, (0, 1): 0.2924901185770751, (1, 0): 0.391304347826087, (1, 1): 0.13043478260869565}
P(Y|X=(np.float64(1.0), np.float64(0.0))) = {(0, 0): 0.10546875, (0, 1): 0.15625, (1, 0): 0.53515625, (1, 1): 0.203125}
[(np.float64(0.0), np.float64(0.0)), (np.float64(0.0), np.float64(1.0)), (np.float64(1.0), np.float64(0.0)), (np.float64(1.0), np.float64(1.0))]
[0.253 0.257 0.256 0.234]
[[0.18577075 0.29249012 0.39130435 0.13043478]
 [0.08171206 0.38521401 0.26459144 0.26848249]
 [0.10546875 0.15625    0.53515625 0.203125  ]
 [0.05982906 0.15384615 0.33760684 0.44871795]]


From the CCP, we can compute the conditional probability of all events $P(A|X_i),A\subseteq\mathcal Y$ for each $X_i$.

In [24]:
_, temp = idc.calculate_subset_probabilities(ccp_array[0,:], Y_nodes)
print(temp)
J = len(temp) # This is the number of all events

Nx = len(ccp_array) # number of unique X values (n for continous X)
p_events = np.zeros((Nx,J))
for i in range(Nx):
  _, p_events[i,:] = idc.calculate_subset_probabilities(ccp_array[i,:], Y_nodes)

# Printing p(A|x) as an Nx-by-J array
print(p_events)

[0.         0.18577075 0.29249012 0.39130435 0.13043478 0.47826087
 0.5770751  0.31620553 0.68379447 0.4229249  0.52173913 0.86956522
 0.60869565 0.70750988 0.81422925 1.        ]
[[0.         0.18577075 0.29249012 0.39130435 0.13043478 0.47826087
  0.5770751  0.31620553 0.68379447 0.4229249  0.52173913 0.86956522
  0.60869565 0.70750988 0.81422925 1.        ]
 [0.         0.08171206 0.38521401 0.26459144 0.26848249 0.46692607
  0.3463035  0.35019455 0.64980545 0.6536965  0.53307393 0.73151751
  0.73540856 0.61478599 0.91828794 1.        ]
 [0.         0.10546875 0.15625    0.53515625 0.203125   0.26171875
  0.640625   0.30859375 0.69140625 0.359375   0.73828125 0.796875
  0.46484375 0.84375    0.89453125 1.        ]
 [0.         0.05982906 0.15384615 0.33760684 0.44871795 0.21367521
  0.3974359  0.50854701 0.49145299 0.6025641  0.78632479 0.55128205
  0.66239316 0.84615385 0.94017094 1.        ]]


# Compute sharp lower bound $\nu_\theta(A|x)$
Now, let's compare $P(A|X_i)$ to the sharp lower bound calculated at some value $\theta$. For the moment, we use the following value as $\theta$ (you can change it to something else.)
- $\beta_1$ = 0.5
- $\beta_2$ = 0.5
- $\Delta_1$ = -0.25
- $\Delta_2$ = -0.5
- $\rho$ = 0.5


As before let's build a model as a graph.

In [25]:
# Define the model
Y_nodes = [(0,0), (0,1), (1,0), (1,1)]
U_nodes = ['a', 'b', 'c', 'd', 'e']
edges = [
    ('a', (0,0)),
    ('b', (0,1)),
    ('c', (1,0)),
    ('d', (1,1)),
    ('e', (0,1)),
    ('e', (1,0))
]
gmodel = idc.BipartiteGraph(Y_nodes, U_nodes, edges)

The next step is to calculate the probability distribution $F_\theta(\cdot|X_i)$ over the $U$-nodes.

In [26]:
import IncompleteDiscreteChoice.examples as idcex

theta_temp = [0.5,0.5,-0.25,-0.5,0.5]
Nu = len(U_nodes)
Ftheta = np.zeros((Nx,Nu))
for i in range(Nx):
  Ftheta[i,:] = idcex.calculate_Ftheta_entrygame(X_supp[i],theta_temp)

# Print Ftheta as Nx-by-Nu array
print(Ftheta)

[[0.33333217 0.19694609 0.25122617 0.19659913 0.02189845]
 [0.22687747 0.32529884 0.14563479 0.28116187 0.02103467]
 [0.22688203 0.10152299 0.40061467 0.25237197 0.01861066]
 [0.16331975 0.18369635 0.25212357 0.37986425 0.02099885]]


Now we are ready to compute the sharp lower bound of CCPs at $\theta$.

In [27]:
nutheta = np.zeros((Nx,J))
for i in range(Nx):
    _,nutheta[i,:] = gmodel.calculate_sharp_lower_bound(Ftheta[i])

# Print lower bound as Nx-by-J array (compare it to p(A|x) above)
print(nutheta)


[[0.         0.33333217 0.19694609 0.25122617 0.19659913 0.53027827
  0.58455834 0.5299313  0.47007071 0.39354523 0.4478253  0.80340289
  0.7268774  0.78115747 0.66666985 1.00000202]
 [0.         0.22687747 0.32529884 0.14563479 0.28116187 0.55217631
  0.37251227 0.50803934 0.49196831 0.60646071 0.42679666 0.71884578
  0.83333818 0.65367413 0.77313017 1.00000764]
 [0.         0.22688203 0.10152299 0.40061467 0.25237197 0.32840503
  0.6274967  0.479254   0.52074832 0.35389496 0.65298663 0.74763035
  0.58077699 0.87986867 0.77312029 1.00000232]
 [0.         0.16331975 0.18369635 0.25212357 0.37986425 0.3470161
  0.41544331 0.543184   0.45681877 0.56356061 0.63198782 0.62013851
  0.72688035 0.79530757 0.83668302 1.00000277]]


Now let's compare the CCP and lower bounds and compute $\hat Q_n(\theta)=\frac{1}{n}\sum_{j}\sum_{x\in\mathcal X}w_x(\nu_\theta(A_j|x)-\hat P_n(A_j|x))_+$,
where $w_x=\sum_{i}1\{X_i=x\}/n$.

In [28]:
difference = nutheta - p_events
diff_pos = np.maximum(difference, 0)
w = np.repeat(Px,J).reshape(Nx,J)
n = len(Y)
Qhat = np.sum(w*diff_pos)/n
print(Qhat)

0.0005659745511206444


# Summary

Let's summarize what we did:
- We computed $\hat p(A|x)$
- We computed $\nu_\theta(A|x)$ at $\theta$
- We computed $\hat Q_n(\theta)$

The IDC library has a wrapper function `idc.calculate_Qhat` to execute the steps avove.

It takes the following objects as inputs
- theta: (parameter)
- [Y,X]: (data)
- gmodel: (class BipartiteGraph)
  - Y-nodes
  - U-nodes
  - Edges
- calculate_Ftheta (function)

You can simply execute the following code.



In [29]:
# If needed uncomment the line below
#!git clone https://github.com/hkaido0718/IncompleteDiscreteChoice.git
import IncompleteDiscreteChoice.idclib as idc
import IncompleteDiscreteChoice.examples as ex
import numpy as np
import gdown

# Download entrygame sample data (same data as above)
url = "https://drive.google.com/uc?id=1cRhMJ8bRhdzy9_agmQ_LkqzlsKRKcthX"
output = "data_entrygame.npz"
gdown.download(url, output, quiet=True)
Data = np.load(output, allow_pickle=True)

# Define the nodes
Y_nodes = [(0,0), (0,1), (1,0), (1,1)]
U_nodes = ['a', 'b', 'c', 'd', 'e']

# Create edges
edges = [
    ('a', (0,0)),
    ('b', (0,1)),
    ('c', (1,0)),
    ('d', (1,1)),
    ('e', (0,1)),
    ('e', (1,0))
]

gmodel = idc.BipartiteGraph(Y_nodes, U_nodes, edges)
theta = [0.5, 0.5, -0.25, -0.5, 0.5]
Y = Data['Y']
X = Data['X']
data = [Y, X]
Qhat = idc.calculate_Qhat(theta, data, gmodel, ex.calculate_Ftheta_entrygame)
print(Qhat)

0.0005659725348612432


You can use `calculate_Qhat` to construct a consistent estimator or construct other test statistics. As an exercise, let's find a minimizer (a point in a consistent set estimator) of this objective function.

In [30]:
from scipy.optimize import differential_evolution

# Assuming the calculate_Qhat function is already defined as provided earlier

# Define the function to minimize
def objective_function(theta):
    return idc.calculate_Qhat(theta, data, gmodel, ex.calculate_Ftheta_entrygame)

# Define the bounds
LB = [-2, -2, -2, -2, 0]
UB = [2, 2, 0, 0, 0.85]
bounds = [(low, high) for low, high in zip(LB, UB)]

# Callback function to print intermediate results
def callback(xk, convergence):
    #print(f"Current theta: {xk}")
    print(f"Current Qhat: {objective_function(xk)}")
    print(f"Convergence: {convergence}")

# Set a seed for replicability
np.random.seed(123)

# Perform the optimization
result = differential_evolution(objective_function, bounds, callback=callback,seed=123)

# Get the optimal theta and the minimum Qhat value
optimal_theta = result.x
min_Qhat = result.fun

print("Optimal theta:", optimal_theta)
print("Minimum Qhat:", min_Qhat)


Constraint violated: Ftheta values below 1e-06. Applying penalty.
Current Qhat: 0.00035026328249376
Convergence: 0.026332234482870615
Constraint violated: Ftheta values below 1e-06. Applying penalty.
Current Qhat: 0.0003502400318506905
Convergence: 0.026493927181952596
Current Qhat: 0.00035023945410772583
Convergence: 0.034645405856145076
Current Qhat: 0.0002614176666387472
Convergence: 0.03868531353879789
Current Qhat: 0.0002572773233581735
Convergence: 0.03821436370375501
Current Qhat: 0.00025726585132830636
Convergence: 0.037791919870780497
Current Qhat: 0.00024860634812482997
Convergence: 0.040607332196319866
Current Qhat: 0.00021857997285072023
Convergence: 0.039661843161200826
Current Qhat: 0.00020939305336332439
Convergence: 0.04293689056891417
Current Qhat: 0.00020419111262286839
Convergence: 0.04445149741938285
Current Qhat: 0.00020419041451740454
Convergence: 0.0459359034979392
Current Qhat: 0.00020420011882787834
Convergence: 0.051723993145326524
Current Qhat: 0.000204187896